In [17]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine

In [18]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [19]:
# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [20]:
# Consulta SQL para obtener datos de la tabla de sedes
query = """
SELECT 
    sede.sede_id,
    sede.nombre AS nombre_sede,
    sede.direccion AS direccion_sede,
    ciudad.nombre AS ciudad_sede,
    departamento.nombre AS departamento_sede
FROM sede 
JOIN ciudad ON sede.ciudad_id = ciudad.ciudad_id
JOIN departamento ON ciudad.departamento_id = departamento.departamento_id
"""

In [21]:
# Leer datos de la fuente
dim_sedes = pd.read_sql(query, fuente_conn)


In [22]:
#Agregar fecha de carga
dim_sedes['saved'] = date.today()

In [15]:
# Cargar en la bodega de datos
dim_sedes.to_sql('dim_sede', bodega_conn, if_exists='replace', index_label='key_dim_sede')

52

In [23]:
# Verificar datos antes de la carga
print("Información del DataFrame:")
print(dim_sedes.info())

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sede_id            52 non-null     int64 
 1   nombre_sede        52 non-null     object
 2   direccion_sede     52 non-null     object
 3   ciudad_sede        52 non-null     object
 4   departamento_sede  52 non-null     object
 5   saved              52 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.6+ KB
None


In [24]:
# Verificaciones adicionales
print("\nNúmero total de sedes:", len(dim_sedes))
print("\nMuestra de los datos:")
print(dim_sedes.head())


Número total de sedes: 52

Muestra de los datos:
   sede_id            nombre_sede               direccion_sede ciudad_sede  \
0       10        FARALLONES /123  Los angeles distrito Latino        CALI   
1       11          REMEDIOZ/ 123  Los angeles distrito Latino        CALI   
2       13       DIME / LOS ROJOS  Los angeles distrito Latino        CALI   
3       14  DESPACHOS / LOS ROJOS  Los angeles distrito Latino        CALI   
4       23  POPAYAN BODEGA 28 / A  Los angeles distrito Latino     POPAYAN   

  departamento_sede       saved  
0   VALLE DEL CAUCA  2024-11-10  
1   VALLE DEL CAUCA  2024-11-10  
2   VALLE DEL CAUCA  2024-11-10  
3   VALLE DEL CAUCA  2024-11-10  
4             CAUCA  2024-11-10  


In [25]:
# Cargar en la bodega de datos 
dim_sedes.to_sql('dim_sede', bodega_conn, if_exists='replace', index_label='key_dim_sede')

52